In [20]:
import cbpro
import pandas as pd

from api import Coinbase_API
from api import Coinbase_secret
from api import Coinbase_pass

api_key = Coinbase_API
api_secret = Coinbase_secret
api_pass = Coinbase_pass

from coinbase.wallet.client import Client

In [153]:
url = 'https://api.pro.coinbase.com'
client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass,
    api_url = url)

total = 0
message = []

accounts = client.get_accounts()
for wallet in accounts:
    if wallet['balance'] > '0.0000000000000000':
        message.append(str(wallet['currency']) + ' ' + str(round(float((wallet['balance'])),4)))
    value = str(wallet['balance']).replace('USD','')
    total += round(float(value), 4)
#message.append('Total Balance: ' + 'USD ' + str(total))
print ('\n'.join( message ))
print('Total Balance: ' + 'USD ' + str(total))

ADA 0.01
AVAX 0.0007
DASH 0.0002
LINK 0.0
SHIB 318167743.0
USD 0.0
USDC 0.0
USDT 0.0029
ZEC 0.0
Total Balance: USD 318167743.0138


In [144]:
class TextWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):  
        self.url = 'wss://ws-feed.prime.coinbase.com'
        self.message_count = 0
    
    def on_message(self,msg):
        self.message_count =+ 1
        msg_type = msg.get('type', None)
        if msg_type == 'ticker':
            time_val = msg.get('time',('-'*27))
            price_val = msg.get('price', None)
            price_val = float(price_val) if price_val is not None else 'None'
            product_id = msg.get('product_id', None)
            
            print(f'{time_val:30} {price_val:3f} {product_id}\tchannel type:{msg_type}')
    
    def on_close(self):
        print(f'<---Websocket connection closed--->\n\tTotal messages: {self.message_count}')

In [146]:
stream = TextWebsocketClient(products=['ETH-USD'],channels=['ticker'])


### Authenticated Client

In [32]:
from api import Coinbase_API
from api import Coinbase_secret
from api import Coinbase_pass

api_key = Coinbase_API
api_secret = Coinbase_secret
api_pass = Coinbase_pass

In [33]:
url = 'https://api.pro.coinbase.com'
client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass,
    api_url = url)

In [34]:
accounts = client.get_accounts()

In [57]:
for acc in accounts:
    currency = acc.get('currency')
    if currency == 'SHIB':
        acc_id = acc.get('id')

In [58]:
acc_history = client.get_account_history(acc_id)

In [60]:
import json

for hist in acc_history:
    print(json.dumps(hist, indent=1))
    